*Author: [Daniel Puente Viejo](https://www.linkedin.com/in/danielpuenteviejo/)*

## **Autogen Multi-Agent Ecosystem: Profile Analysis with Free LLMs**

<img src="imgs/cover_page.png" height="350"> 

Practical example of how to create a multi-agent project with Autogen and completely free LLMs.

### **Index:**

- <a href='#1'><ins>1 - Libraries</ins></a>
- <a href='#2'><ins>2 - Model configuration</ins></a>
- <a href='#3'><ins>3 - Termination policy</ins></a>
- <a href='#4'><ins>4 - Agents</ins></a>
    - <a href='#4.1'><ins>4.1 - User Proxy</ins></a>
    - <a href='#4.2'><ins>4.2 - Principal Agents</ins></a>
- <a href='#5'><ins>5 - Tools</ins></a>
- <a href='#6'><ins>6 - Agent order</ins></a>
- <a href='#7'><ins>7 - Multi-agent ecosystem</ins></a>
- <a href='#8'><ins>8 - Execution</ins></a>

### <a id='1' style="color: skyblue;">**1 - Libraries**</a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import autogen
from autogen import UserProxyAgent, register_function, AssistantAgent

import os
from dotenv import load_dotenv
load_dotenv()

True

### <a id='2' style="color: skyblue;">**2 - Model configuration**</a>   


In this case, we will create two configurations: one using `llama-3.1-8b-instant` and another using `gemma2-9b-it`. Take a look to the rest of the available models [here](https://console.groq.com/docs/models).

In [2]:
# llama 3.1 instant config
llama31_config = {
    "config_list": [
        {
            "model": "llama-3.1-8b-instant",
            "api_key": os.environ.get("GROQ_API_KEY"),
            "api_type": "groq"
        }
    ]
}

# gemma2 config
gemma2_config = {
    "config_list": [
        {
            "model": "gemma2-9b-it",
            "api_key": os.environ.get("GROQ_API_KEY"),
            "api_type": "groq"
        }
    ]
}

### <a id='3' style="color: skyblue;">**3 - Termination policy**</a>     


Autogen requires a mechanism to determine when a process has reached its conclusion. To achieve this, we need to analyze the result and identify a specific pattern. A recommended approach is to program agents to return the word "TERMINATE" when they conclude that the query has been successfully resolved. For this purpose, we define the following function:

In [3]:
def termination_message(msg: str) -> bool:
    content = msg.get("content", "").lower()
    return "terminate" in content

### <a id='4' style="color: skyblue;">**4 - Agents**</a>


#### <a id='4.1' style="color: azure;">**4.1 - User Proxy**</a>   

Let's begin with the UserProxyAgent. It acts as an intermediary, representing a human user within a multi-agent system. Its primary function is to execute code and provide feedback to other agents, enabling human-in-the-loop interactions during automated processes. In this case, it is only responsible for executing tools and returning the results to the appropriate agent.

In [4]:
# The user proxy agent that executes tool calls.
user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

#### <a id='4.2' style="color: azure;">**4.2 - Principal Agents**</a>   

The idea is straightforward: it involves a group of agents designed to analyze a CV or profile, each specializing in a specific task. This group will consist of four agents:

* CV Analyzer 📊: This agent specializes in analyzing the input CV or profile document. Its task is to extract the most relevant information from the document, such as professional experience, education, and notable achievements, providing a structured summary of the candidate's profile.
* Skills Extractor 🔧: This agent is responsible for extracting the key skills from the CV. It identifies competencies such as programming languages, technical proficiencies, or soft skills and returns them in a concise, comma-separated list for easy reference.
* Courses Recommender 👨‍🎓: This agent matches the extracted skills with a database of available courses. Its goal is to recommend courses that can help the candidate enhance or expand their skill set, providing tailored suggestions based on their profile.
* Resolution Checker 🎯: The role of this agent is to determine whether the user's query has been fully resolved. It reviews the outputs of the other agents and, if the task is completed, it is responsible for forming the final answer.

In [5]:
## -- Agent 1: CV Analyser --
cv_analyser = AssistantAgent(
    name="CV Analyser",
    description="It analyzes the input it giving the most relevant information of the document.",
    system_message="""You must return an analysis of the input file.
    Other details for your answer:
    - Do not responde any piece of code.
    - Do not recommend courses just analyse the input file.""",
    llm_config=llama31_config,
)

## -- Agent 2: Skills Extractor --
skills_extractor = AssistantAgent(
    name="Skills Extractor",
    description="Returns a list of the candidates skills",
    system_message="""Your task is to extract the main skills from the input received.
    You must return the skills as a comma-separated list. For example: 'Python, Java, SQL'. Do not add more information, just the list of skills in the specified format.
    Other details for your answer:
    - Do not responde any piece of code.
    - Do not recommend courses just return the skills as a comma-separated list of the input file.""",
    llm_config=llama31_config,
)

## -- Agent 3: Courses Recommender --
courses_recommender = AssistantAgent(
    name="Courses recommender",
    description="The goal is to recommend courses. Read the courses from `data/courses.txt` and return the courses that match the skills of the input file.",
    system_message="""The goal is to recommend courses, for that follow these steps:
    1) Read the list of courses from `data/courses.txt` (use `read_txt_courses` tool).
    2) Combine the list of courses with the profile of the candidate.
    3) Return the courses that match the skills of the candidate.
    Other details for your answer: Do not responde any piece of code.""",
    llm_config=llama31_config,
)

# -- Agent 4: Resolution Checker Agent --
resolution_checker = AssistantAgent(
    name="Resolution Checker Agent",
    description="Checks if the user's query has been resolved. If resolved, respond with 'TERMINATE', otherwise respond with 'CONTINUE'. Do not give more information just the word 'CONTINUE' or 'TERMINATE'.",
    system_message="""You should respond with 'TERMINATE' if the query has been resolved. Otherwise, respond with 'CONTINUE'. Do not add more information, just the word 'CONTINUE'.
    Try to be resolute, do not reply 'TERMINATE' if the initial query has not been resolved.
    **IMPORTANT**: Only respond with 'TERMINATE' if the query has been resolved, otherwise respond with 'CONTINUE'. If the answer is 'TERMINATE', you must answer 'TERMINATE' + generate a complete answer to the user's query considering all the information given by all the agents to generate the final answer.""",
    llm_config=gemma2_config,
    is_termination_msg=termination_message,
)

### <a id='5' style="color: skyblue;">**5 - Tools**</a>


This is a simple Python function that reads a `.txt` file. The use of type annotations is important to clearly define the expected input and output of the function.

In [6]:
# Tool: Read CV from File
def read_txt(path: str) -> str:
    with open(path, "r") as file:
        return file.read()

The next step is to configure each agent to allow the use of this tool. The tool is executed by the User Proxy, so we need to specify the caller, which is the agent that can access the tool, and the executor, which will be the User Proxy. Additionally, we must provide a name and description for the tool.

In [7]:
# Register the read_txt function with the assistant agent.
register_function(
    read_txt,
    caller=cv_analyser,
    executor=user_proxy,
    name="read_txt_cv_analyse",
    description="Read the profile of the user",
)

register_function(
    read_txt,
    caller=courses_recommender,
    executor=user_proxy,
    name="read_txt_courses",
    description="Read available courses and/or the profile of the user",
)

register_function(
    read_txt,
    caller=skills_extractor,
    executor=user_proxy,
    name="read_txt_cv_skills",
    description="Read the profile of the user",
)

### <a id='6' style="color: skyblue;">**6 - Agent order**</a>  

There are different ways to manage a group of agents. Let's review the most common conversational patterns:

* **Round Robin:** Agents follow a predefined order, regardless of the query. The system executes them sequentially according to this order.
* **Random**: The speaker selection is random. This approach is not highly recommended due to its lack of structure.
* **Manual**: When a query is made, the system starts, and agent selection is performed manually by a human. After one agent completes its task, the system asks the user to specify the next agent, continuing until the query is considered resolved.
* **Auto**: An LLM decides the next speaker based on the context of the conversation and the description of each agent.
* **Hybrid**: A combination of manual and automated approaches. You can establish a set of rules to allow the LLM manager to select the next agent or specify which agents will act depending on predefined conditions.
 

Of course, there are more complex ways to manage agent ecosystems, but we will focus on these examples to keep the article concise and help you understand the concept. 

We will focus on the last type, **hybrid**. To implement this, we need to create a custom function:

In [8]:
# Create a GroupChatManager to oversee the conversation
def custom_speaker_selection(last_speaker, groupchat):
    """
    Function to change the agent selection logic and customize the speaker selection.

    Args:
    last_speaker (Agent): The last agent that spoke.
    groupchat (GroupChat): The group chat object.

    Returns:
    Agent: The agent that will speak next.
    """
    agents_to_check = ["cv analyser", "skills extractor", "courses recommender"]

    # Extract the last and penultimate messages
    last_message = groupchat.messages[-1]
    penultimate_message = groupchat.messages[-2] if len(groupchat.messages) > 1 else None

    # Extract the names of the last and penultimate speakers
    last_speaker_name = last_speaker.name.lower()
    penultimate_speaker_name = penultimate_message['name'] if penultimate_message else None

    ### If the agent has made a tool call, call the 'User'
    if last_message.get("tool_calls", ""):
        return next(agent for agent in groupchat.agents if agent.name == "User")

    ### If the last speaker was the 'User' and the penultimate speaker was not the user or the Resolution Checker Agent, call the penultimate speaker
    ### Example: Skills Extractor makes use of a tool, so it call the 'User' agent. The 'User' agent then calls the 'Skills Extractor' to continue with the task it was doing.
    if last_speaker_name == "user" and penultimate_speaker_name and penultimate_speaker_name.lower() not in ["user", "resolution checker agent"]:
        return next(agent for agent in groupchat.agents if agent.name == penultimate_speaker_name)

    ### The an agent has given an answer, check if the quey is complete. For that call the 'Resolution Checker Agent'
    ### Example: The 'Courses Recommender' agent has given an answer, so the 'Resolution Checker Agent' is called to check if the query is complete.
    if last_speaker_name in agents_to_check:
        return next(agent for agent in groupchat.agents if agent.name == "Resolution Checker Agent")

    ### If no condition is met, leave the automatic configuration so that the next agent selection is based on an LLM.
    return 'auto'

### <a id='7' style="color: skyblue;">**7 - Multi-agent ecosystem**</a>   

The final step before interacting with the system is to create the engine that handles everything we have constructed.

The group chat specifies the agents participating in the system, how to select the next agent, whether an agent can speak consecutively or not, and the focus criteria for agent selection. 

The manager can be defined as the entity that will manage the whole system. It requires the previously defined group chat, specifies that code execution does not use Docker, determines the LLM configuration for agent selection, and defines the termination message to signal when the process is complete.

In [9]:
# Create the group chat
groupchat = autogen.GroupChat(
    agents=[
        user_proxy,
        cv_analyser,
        skills_extractor,
        courses_recommender,
        resolution_checker,
    ],
    messages=[],
    allow_repeat_speaker=True,
    speaker_selection_method=custom_speaker_selection, # Other options are: "round_robin", "random", "manual", "auto"
    role_for_select_speaker_messages="system"
)

# Finally, create the group chat manager
manager = autogen.GroupChatManager(
    groupchat=groupchat,
    code_execution_config={"use_docker": False},
    llm_config=gemma2_config,
    is_termination_msg=termination_message,
)

### <a id='8' style="color: skyblue;">**8 - Execution**</a>

First, we define the file path variable for the CV. Next, we create the message specifying what we want the system to do. Some example messages are provided to illustrate different functionalities of the system.  

Finally, we initiate the chat using the User Proxy, passing the manager to control the conversation, the message to be processed, and the maximum number of turns. Setting a high value for max_turns ensures that even if the termination criteria are not met, the conversation will eventually end without being cut off prematurely.

In [13]:
# File path for the CV
cv_file_path = "data/cv.txt"

# Define the message to be sent to the manager
message = f"Recommend courses for the profile {cv_file_path}"
# message = f"Analyse the following profile: {cv_file_path}."
# message = f"Extract skills in comma-separated format based on the following profile: {cv_file_path}."
# message = f"Recommends courses for the profile {cv_file_path}. Return also the profile skills in a list."

chat_result = user_proxy.initiate_chat(
    recipient=manager,
    message=message,
    max_turns=30,
)

User (to chat_manager):

Recommend courses for the profile data/cv.txt

--------------------------------------------------------------------------------

Next speaker: Courses recommender

Courses recommender (to chat_manager):

***** Suggested tool call (call_44re): read_txt_courses *****
Arguments: 
{"path":"data/cv.txt"}
*************************************************************
***** Suggested tool call (call_51mv): read_txt_courses *****
Arguments: 
{"path":"data/courses.txt"}
*************************************************************

--------------------------------------------------------------------------------

Next speaker: User


>>>>>>>> EXECUTING FUNCTION read_txt_courses...

>>>>>>>> EXECUTING FUNCTION read_txt_courses...
User (to chat_manager):

User (to chat_manager):

***** Response from calling tool (call_44re) *****
Daniel Puente Viejo
+34 638 097 547
daniel.puenteviejo@gmail.com
https://github.com/DanielPuentee | https://www.linkedin.com/in/danielpuenteviejo 

In [15]:
# See the agent order
agent_order = [x['name'] for x in chat_result.chat_history]
print(agent_order)

['User', 'Courses recommender', 'User', 'Courses recommender', 'Resolution Checker Agent']


In [14]:
# Show the last result
print(chat_result.chat_history[-1]['content'])

TERMINATE 

Based on the provided profile, here are some course recommendations tailored to Daniel Puente Viejo's experience and interests:

**Deepening his Expertise:**

* **Deep Learning Specialization (DeepLearning.AI):** Given his experience leading the design of Repsol's cross AI Multi-Agent platform and LLMOps platform (oriented towards Azure solutions), this specialization will provide advanced knowledge in deep learning architectures and applications.  
* **Machine Learning for Healthcare (Stanford University):**  His experience at NTT Data working on generative AI for the Spanish Ministry of Digital Transformation suggests an interest in applying AI to real-world problems.  This course could further develop his skills in this area.
* **Generative Adversarial Networks (GANs) Specialization (Coursera):**  His work on generative AI platforms and his experience using diffusion algorithms for image description point to a strong interest in generative models.  This specialization wi

---